# Global Invoicing: Extracting Data

## Setup Prophet Database Connection

In [1]:
# Install pyodbc driver
!pip install pyodbc

In [2]:
import pyodbc
import os
import dotenv

dotenv.load_dotenv()

server = os.getenv("PROPHET_SERVER")
database = os.getenv("PROPHET_DATABASE")
username = os.getenv("PROPHET_USERNAME")
password = os.getenv("PROPHET_PASSWORD")
driver = '{ODBC Driver 17 for SQL Server}'

start_date = '2021-01-01'
end_date = '2021-09-01'

cnxn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = cnxn.cursor()


In [3]:
# Check connection
cursor.execute("SELECT @@version;")
row = cursor.fetchone()
while row:
    print(row[0])
    row = cursor.fetchone()

Microsoft SQL Server 2019 (RTM-CU8-GDR) (KB4583459) - 15.0.4083.2 (X64) 
	Nov  2 2020 18:35:09 
	Copyright (C) 2019 Microsoft Corporation
	Standard Edition (64-bit) on Windows Server 2019 Standard 10.0 <X64> (Build 17763: ) (Hypervisor)



## Getting Data

### Jobs

In [4]:
cursor.execute("""
            SELECT DISTINCT 
            lotdet_nl.jobnum,
            lothed_nl.ponum
            FROM palstk_nl
            LEFT JOIN lotdet_nl ON palstk_nl.lotdetid=lotdet_nl.lotdetid
            LEFT JOIN jobbug_nl ON lotdet_nl.jobnum=jobbug_nl.jobnum
            LEFT JOIN lothed_nl ON lotdet_nl.lotnum=lothed_nl.lotnum
            LEFT JOIN poffil_nl ON lothed_nl.ponum=poffil_nl.ponum
            WHERE jobbug_nl.bookdate BETWEEN '{}' AND '{}'
            AND (lothed_nl.supcode = 'GLFRE' OR lothed_nl.supcode = 'GPPRO')
            AND upper(poffil_nl.narrative) NOT LIKE '%DUNNES%'
            AND upper(poffil_nl.narrative) NOT LIKE '%ICELAND%'
            AND upper(poffil_nl.narrative) NOT LIKE '%LIDL%'
            AND jobbug_nl.status != 25
            AND jobbug_nl.status != 30
            AND jobbug_nl.status != 50
            AND palstk_nl.recqty != 0;
""".format(start_date, end_date))

jobs = []

row = cursor.fetchone()
while row:
    jobs.append(str(row[0]))
    row = cursor.fetchone()

### Orders

In [5]:
import csv

csv_file_path = '../data/raw/orders.csv'

cursor.execute("""
    SELECT palord_nl.palordid,
    palord_nl.palstkid,
    palord_nl.qty,
    palord_nl.orddetid,
    poffil_nl.ponum,
    ordhed_nl.ordnum,
    ordhed_nl.delcustcode,
    ordhed_nl.deldate,
    custac_nl.name,
    lotdet_nl.prodnum,
    lotdet_nl.jobnum,
    spdfil_nl.mark,
    prdall_nl.descr,
    spdfil_nl.countsize,
    orddet_nl.uomtradingqty,
    orddet_user_nl.text1 as sodusseldorfs
    FROM palord_nl
    LEFT JOIN palstk_nl ON palord_nl.palstkid=palstk_nl.palstkid
    LEFT JOIN orddet_nl ON palord_nl.orddetid=orddet_nl.orddetid
    LEFT JOIN ordhed_nl ON orddet_nl.ordhedid=ordhed_nl.ordhedid
    LEFT JOIN custac_nl ON ordhed_nl.custcode=custac_nl.custcode
    LEFT JOIN lotdet_nl ON palstk_nl.lotdetid=lotdet_nl.lotdetid
    LEFT JOIN lothed_nl ON lotdet_nl.lotnum=lothed_nl.lotnum
    LEFT JOIN poffil_nl ON lothed_nl.ponum=poffil_nl.ponum
    LEFT JOIN spdfil_nl ON orddet_nl.prodnum=spdfil_nl.prodnum
    LEFT JOIN prdall_nl ON spdfil_nl.mascode=prdall_nl.mascode
    LEFT JOIN orddet_user_nl ON orddet_nl.orddetid=orddet_user_nl.orddetid
    LEFT JOIN palfil_nl ON palstk_nl.palfilid=palfil_nl.palfilid
    LEFT JOIN palfil_user_nl ON palfil_nl.palfilid=palfil_user_nl.palfilid
    WHERE ordhed_nl.deldate BETWEEN '{}' AND '{}'
    AND (ordhed_nl.delcustcode = 'GLALM' OR ordhed_nl.delcustcode = 'GLALN')
    AND ordhed_nl.custcode != 'PPACK'
    AND ordhed_nl.custcode != 'GLOB'
    AND ordhed_nl.dp != '99'
    AND orddet_nl.uomtradingqty != 0
    AND palord_nl.qty != 0
    ORDER BY ordhed_nl.deldate ASC, custac_nl.name ASC;
""".format(start_date, '2021-10-01'))

rows = cursor.fetchall()

if rows:
    result = list()

    column_names = list()
    for i in cursor.description:
        column_names.append(i[0])
    
    result.append(column_names)

    for row in rows:
        result.append(row)

    with open(csv_file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in result:
            csvwriter.writerow(row)


### Job Ingredients

In [6]:
csv_file_path = '../data/raw/ingredients.csv'

result = list()

query = """
    SELECT
    bomlot_nl.palstkid,
    bomlot_nl.actqty,
    bomlot_nl.actweight,
    boming_nl.prodnum,
    prdall_nl.descr,
    spdfil_nl.mark,
    spdfil_nl.countsize,
    lothed_nl.ponum,
    palstk_nl.palfilid,
    jobbug_nl.jobnum
    FROM bomlot_nl
    LEFT JOIN boming_nl ON bomlot_nl.bomingid=boming_nl.bomingid
    LEFT JOIN jobbom_nl ON boming_nl.jobbomid=jobbom_nl.jobbomid
    LEFT JOIN jobbug_nl ON jobbom_nl.jobnum=jobbug_nl.jobnum
    LEFT JOIN spdfil_nl ON boming_nl.prodnum=spdfil_nl.prodnum
    LEFT JOIN prdall_nl ON spdfil_nl.mascode=prdall_nl.mascode
    LEFT JOIN palstk_nl ON bomlot_nl.palstkid=palstk_nl.palstkid
    LEFT JOIN lotdet_nl ON palstk_nl.lotdetid=lotdet_nl.lotdetid
    LEFT JOIN lothed_nl ON lotdet_nl.lotnum=lothed_nl.lotnum
    WHERE jobbug_nl.jobnum IN ({})
    AND spdfil_nl.groupnum='2'
    AND bomlot_nl.actqty != '0';
""".format(", ".join(jobs))

cursor.execute(query)

rows = cursor.fetchall()

if rows:
    column_names = list()
    for i in cursor.description:
        column_names.append(i[0])
    
    result.append(column_names)

    for row in rows:
        result.append(row)
    

with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in result:
        csvwriter.writerow(row)

### Finished

In [7]:
csv_file_path = '../data/raw/finished.csv'

result = list()

query = """
    SELECT
    palstk_nl.palstkid,
    palstk_nl.palfilid,
    lotdet_nl.jobnum,
    palstk_nl.recqty,
    palstk_nl.weight,
    lotdet_nl.prodnum,
    lotdet_nl.class,
    prdall_nl.descr,
    spdfil_nl.mark,
    spdfil_nl.qtyouter,
    jobbug_nl.buildcatnum,
    jobbug_nl.reqdate,
    buildcatdescr_nl.buildcatdescr,
    spdfil_nl.countsize,
    jobbug_nl.comment4 as freejob
    FROM palstk_nl
    LEFT JOIN lotdet_nl ON palstk_nl.lotdetid=lotdet_nl.lotdetid
    LEFT JOIN spdfil_nl ON lotdet_nl.prodnum=spdfil_nl.prodnum
    LEFT JOIN prdall_nl ON spdfil_nl.mascode=prdall_nl.mascode
    LEFT JOIN jobbug_nl ON lotdet_nl.jobnum=jobbug_nl.jobnum
    LEFT JOIN buildcatdescr_nl ON jobbug_nl.buildcatnum=buildcatdescr_nl.buildcatnum
    WHERE lotdet_nl.jobnum in ({})
    AND palstk_nl.recqty != '0'
    AND lotdet_nl.class='1';
""".format(", ".join(jobs))

cursor.execute(query)

rows = cursor.fetchall()

if rows:
    column_names = list()
    for i in cursor.description:
        column_names.append(i[0])

    result.append(column_names)

    for row in rows:
        result.append(row)


with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in result:
        csvwriter.writerow(row)

### Packaging

In [8]:
csv_file_path = '../data/raw/packaging.csv'

result = list()

query = """
    SELECT
    bomlot_nl.palstkid,
    bomlot_nl.actqty,
    boming_nl.prodnum,
    prdall_nl.descr,
    spdfil_nl.mark,
    bomlot_nl.price
    FROM bomlot_nl
    LEFT JOIN boming_nl ON bomlot_nl.bomingid=boming_nl.bomingid
    LEFT JOIN jobbom_nl ON boming_nl.jobbomid=jobbom_nl.jobbomid
    LEFT JOIN jobbug_nl ON jobbom_nl.jobnum=jobbug_nl.jobnum
    LEFT JOIN spdfil_nl ON boming_nl.prodnum=spdfil_nl.prodnum
    LEFT JOIN prdall_nl ON spdfil_nl.mascode=prdall_nl.mascode
    WHERE jobbug_nl.jobnum in ({})
    AND spdfil_nl.groupnum='5'
    AND bomlot_nl.actqty!=0;
""".format(", ".join(jobs))

cursor.execute(query)

rows = cursor.fetchall()

if rows:
    column_names = list()
    for i in cursor.description:
        column_names.append(i[0])
    
    result.append(column_names)

    for row in rows:
        result.append(row)
    

with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in result:
        csvwriter.writerow(row)

### Resources

In [9]:
csv_file_path = '../data/raw/resources.csv'

result = list()

query = """
    SELECT rscdet_nl.rescode,
    jobbom_nl.jobnum,
    bomres_nl.linevalue
    FROM bomres_nl
    LEFT JOIN rscdet_nl ON bomres_nl.bomresid=rscdet_nl.bomresid
    LEFT JOIN jobbom_nl ON bomres_nl.jobbomid=jobbom_nl.jobbomid
    WHERE jobbom_nl.jobnum IN ({})
    AND bomres_nl.linevalue != 0;
""".format(", ".join(jobs))

cursor.execute(query)

rows = cursor.fetchall()

if rows:
    column_names = list()
    for i in cursor.description:
        column_names.append(i[0])
    
    result.append(column_names)

    for row in rows:
        result.append(row)
    

with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in result:
        csvwriter.writerow(row)

### Waste

In [10]:
csv_file_path = '../data/raw/waste.csv'

result = list()

query = """
    SELECT MAX(palord_nl.palordid) as palordid,
    COUNT(palord_nl.palordid) AS ordcount,
    palord_nl.palstkid AS palstkid,
    SUM(palord_nl.qty) AS qty,
    MAX(palord_nl.orddetid) AS orddetid,
    MAX(ordhed_nl.ordnum) AS ordnum,
    MAX(poffil_nl.ponum) AS ponum,
    MAX(ordhed_nl.delcustcode) AS delcustcode,
    MAX(ordhed_nl.deldate) AS deldate,
    MAX(custac_nl.name) AS name,
    MAX(lotdet_nl.prodnum) AS prodnum,
    MAX(lotdet_nl.jobnum) AS jobnum,
    MAX(spdfil_nl.mark) AS mark,
    MAX(prdall_nl.descr) AS descr,
    MAX(spdfil_nl.countsize) AS countsize,
    MAX(spdfil_nl.wgtouter) AS wgtouter,
    AVG(palstk_nl.weight) as weight
    FROM palord_nl
    LEFT JOIN palstk_nl ON palord_nl.palstkid=palstk_nl.palstkid
    LEFT JOIN orddet_nl ON palord_nl.orddetid=orddet_nl.orddetid
    LEFT JOIN ordhed_nl ON orddet_nl.ordhedid=ordhed_nl.ordhedid
    LEFT JOIN custac_nl ON ordhed_nl.custcode=custac_nl.custcode
    LEFT JOIN lotdet_nl ON palstk_nl.lotdetid=lotdet_nl.lotdetid
    LEFT JOIN lothed_nl ON lotdet_nl.lotnum=lothed_nl.lotnum
    LEFT JOIN poffil_nl ON lothed_nl.ponum=poffil_nl.ponum
    LEFT JOIN spdfil_nl ON orddet_nl.prodnum=spdfil_nl.prodnum
    LEFT JOIN prdall_nl ON spdfil_nl.mascode=prdall_nl.mascode
    LEFT JOIN orddet_user_nl ON orddet_nl.orddetid=orddet_user_nl.orddetid
    WHERE lotdet_nl.jobnum IN ({})
    AND (
        ordhed_nl.custcode = 'GLOB'
        OR ordhed_nl.custcode = 'WPACK'
    )
    AND orddet_nl.uomtradingqty != 0
    GROUP BY palord_nl.palstkid
    ORDER BY deldate ASC, name ASC;
""".format(", ".join(jobs))

cursor.execute(query)

rows = cursor.fetchall()

if rows:
    column_names = list()
    for i in cursor.description:
        column_names.append(i[0])
    
    result.append(column_names)

    for row in rows:
        result.append(row)
    

with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in result:
        csvwriter.writerow(row)